In [21]:
import json
import requests
import re

use_tor = False

## GDELT QUERYING:

In [22]:
def api_url_constructor(query,
                        mode='artlist',
                        format='json',
                        records=250,
                        start_date=20200101000000,
                        end_date=20220131235959):
    query = query.replace(' ', '%20')
    url_head = "https://api.gdeltproject.org/api/v2/doc/doc?"
    body = f"query={query}&mode={mode}&format={format}&maxrecords={records}"
    params = f"&startdatetime={start_date}&enddatetime={end_date}"
    trans = "&trans=googtrans"
    
    full_url = url_head + body + params + trans
    
    return full_url


def tor_request(url_input):
    '''
    Solo si tienes TOR habilitado en tu ordenador
    https://medium.com/@jasonrigden/using-tor-with-the-python-request-library-79015b2606cb 
    
    Usa la red TOR para hacer peticiones desde IPs aleatorias simulando navegadores.
    Esto anula los límites de llamadas a APIs gratuitas, que suelen fijarse en IP.
    '''
    tor_proxy = {
        'http': 'socks5://127.0.0.1:9050',
        'https': 'socks5://127.0.0.1:9050'
    }

    session = requests.session()
    session.proxies = tor_proxy
    session.headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    socks.set_default_proxy(socks.SOCKS5, "127.0.0.1", 9050)
    socket.socket = socks.socksocket

    r = session.get(url_input)
    return r


if use_tor:
    socks.set_default_proxy(socks.SOCKS5, "127.0.0.1", 9050)
    socket.socket = socks.socksocket

def query_cleaner(query):
    ''' 
    La API de GDELT no acepta caracteres especiales en la query.
    Los sustituimos por espacios.
    '''
    patrones = r'[^\w\s\"\(\)]+'
    query = re.sub(patrones, ' ', query)
    
    # Terminos de denominacion empresarial que no son utiles:
    terminos_eliminar = r' Inc\.| Corp\.| Ltd\.| PLC| Co\.| S\.A\.|& KGaA|& KgaA|Co\., Ltd\.|P L C|\(publ\)|\.Com|\.com|A/S|AB|AE|AG|AS|ASA|Abp|CORP|Co|Corp|GmbH & KGaA|Inc|Inc\.|KGaA|LLC|LP|LPG|LTD|Ltd|MFG|NL|NV|Oyj|PLC|Plc|RL|SA|SE|SGPS|SpA|plc|rp'

    query = re.sub(terminos_eliminar,
                    '',
                    query,
                    flags=re.IGNORECASE)
    
    # Quitamos dobles espacios:
    query = re.sub(r'\s+', ' ', query)
    return query


def name_amplifier(name):
    '''
    A veces los nombres tienen demasiadas palabras.
    Esto hace la query muy específica y puede obviar resultados.
    Añadimos permutaciones de las palabras del nombre con 'OR'
    '''
    nombre_query = name # Por defecto, el nombre original
    
    if name.split(' ')>2:
        nombre_corto = ''
        nombre_corto = ' '.join(name.split()[:3])
        nombre_corto_2 = ' '.join(name.split()[:2])
        # Unimos con 'OR' los diferentes nombres:
        nombre_query = f'("{name}" OR "{nombre_corto}" OR "{nombre_corto_2}")'
    
    return nombre_query


def query_completion(query, positives, negatives):
    '''
    Función que recibe una lista de positivos(tuplas) y otra de negativos.
    Los positivos se añaden a la query como:
    'AND ("positivo1" OR "positivo1_alt" OR ...)'
    'AND ("positivo2" OR "positivo2_alt" OR ...)'
    Los negativos se añaden como 'AND -negativo1 AND -negativo2 ...'
    '''
    # Si hay positivos, los añadimos a la query:
    if positives:
        positives = ''
        for p in positives:
            # Si solo hay un elemento en la tupla, no hace falta añadir OR:
            positive_to_add = p[0]
            
            # Si hay más de un elemento, añadimos OR:
            if len(p)>1:
                for i in p[1:]:
                    positive_to_add += f' OR "{i}"'
            
            # El formato es AND ("positivo1" OR "positivo1_alt" OR ...)
            positives += f'AND ({positive_to_add})'
    

In [24]:
positives = [('Bayer', 'Bayer AG', 'Bayerische Motoren Werke AG', 'BMW AG', 'BMW'),
             ('Acerinox', 'Acerinox S.A.', 'Acerinox SA')]

In [29]:
if positives:
    positives_query = ''
    for p in positives:
        # Si solo hay un elemento en la tupla, no hace falta añadir OR:
        positive_to_add = f'"{p[0]}"'
        
        # Si hay más de un elemento, añadimos OR:
        if len(p)>1:
            for i in p[1:]:
                positive_to_add += f' OR "{i}"'
        
        # El formato es AND ("positivo1" OR "positivo1_alt" OR ...)
        positives_query += f'AND ({positive_to_add})'

positives_query

'AND ("Bayer" OR "Bayer AG" OR "Bayerische Motoren Werke AG" OR "BMW AG" OR "BMW")AND ("Acerinox" OR "Acerinox S.A." OR "Acerinox SA")'

In [20]:
api_url_constructor('Acerinox')

'https://api.gdeltproject.org/api/v2/doc/doc?query=Acerinox&mode=artlist&format=json&maxrecords=250&startdatetime=20200101000000&enddatetime=20220131235959&trans=googtrans'

In [ ]:
requests.request

In [15]:
respuesta = requests.get(url=api_url_constructor('Acerinox'))

In [18]:
respuesta.json()

{'articles': [{'url': 'https://www.marketwatch.com/articles/acerinox-stock-price-outlook-51639175858',
   'url_mobile': '',
   'title': 'Acerinox Stock : Huge Demand for Appliances , Autos Could Help Shares 50 % ',
   'seendate': '20211212T173000Z',
   'socialimage': 'https://images.barrons.com/im-448699/social',
   'domain': 'marketwatch.com',
   'language': 'English',
   'sourcecountry': 'United States'},
  {'url': 'https://www.estrategiasdeinversion.com/actualidad/dividendos/acerinox-aprueba-una-recompra-de-acciones-del-n-492947',
   'url_mobile': 'https://www.estrategiasdeinversion.com/amp/acerinox-aprueba-una-recompra-de-acciones-del-n-492947',
   'title': 'Acerinox aprueba una recompra de acciones del 4 % de la compaa',
   'seendate': '20211220T114500Z',
   'socialimage': 'https://www.estrategiasdeinversion.com/images/amp/bolsa-madrid-7.jpg',
   'domain': 'estrategiasdeinversion.com',
   'language': 'Spanish',
   'sourcecountry': 'Spain'},
  {'url': 'https://www.marketwatch.com/a

https://medium.com/@jasonrigden/using-tor-with-the-python-request-library-79015b2606cb